In [1]:
import pandas as pd
data = pd.read_csv("data_all.csv")
len(data)

10170

In [2]:
location = pd.read_csv("data/학교기본정보(초)_서울특별시교육청.csv")
location.columns

Index(['시도교육청', '교육지원청', '지역', '정보공시 학교코드', '학교명', '학교급코드', '설립구분', '학교특성',
       '분교여부', '설립유형', '주야구분', '개교기념일', '설립일', '법정동코드', '주소내역', '상세주소내역',
       '우편번호', '학교도로명 우편번호', '학교도로명 주소', '학교도로명 상세주소', '위도', '경도', '전화번호',
       '팩스번호', '홈페이지 주소', '남녀공학 구분', '폐교여부', '폐교일자', '휴교여부'],
      dtype='object')

학교정보 + 위치

In [3]:
location = location[['정보공시 학교코드','위도','경도']]
location

,정보공시 학교코드,위도,경도
0,S000003511,37.490739,127.015424
1,S000003563,37.577791,127.002875
2,S010000737,37.481618,127.058738
3,S010000738,37.486214,127.057742
4,S010000739,37.486805,127.069977
...,...,...,...
609,S010006432,37.476731,126.823929
610,S010006433,37.498364,127.101956
611,S010006476,37.608890,127.105540
612,S010006510,37.481026,127.062043


In [4]:
merge_data = pd.merge(data, location, on = '정보공시 학교코드',how = 'inner')
merge_data.columns

Index(['년도', '지역', '정보공시 학교코드', '학교명', '입학생_남', '입학생_여', '입학생수', '1학년 학급수',
       '1학년 학생수', '1학년 학급당 학생수', '2학년 학급수', '2학년 학생수', '2학년 학급당 학생수',
       '3학년 학급수', '3학년 학생수', '3학년 학급당 학생수', '4학년 학급수', '4학년 학생수',
       '4학년 학급당 학생수', '5학년 학급수', '5학년 학생수', '5학년 학급당 학생수', '6학년 학급수',
       '6학년 학생수', '6학년 학급당 학생수', '특수학급 학급수', '특수학급 학생수', '특수학급 학급당 학생수',
       '학급수(계)', '학생수(계)', '학급당 학생수(계)', '교사수', '수업교원 1인당 학생수', '위도', '경도'],
      dtype='object')

폐교학교의 지정

In [5]:
years = sorted(merge_data['년도'].unique())
last_year = years[-1]  # 2025년

closed_list = []

for year in years[:-1]:  # 마지막 연도 제외
    data_curr = merge_data[merge_data['년도'] == year].reset_index(drop=True)
    
    for _, row in data_curr.iterrows():
        school_code = row['정보공시 학교코드']
        
        # 해당 학교가 이후 데이터에 한 번도 등장하지 않으면 폐교로 간주
        appears_later = merge_data[
            (merge_data['년도'] > year) & (merge_data['정보공시 학교코드'] == school_code)
        ]
        
        if appears_later.empty:  # 이후에 안 나타나면 폐교
            closed_school = row.copy()
            closed_school['폐교연도'] = year + 1  # 최초로 사라진 다음 연도
            closed_list.append(closed_school)

# 결과 DataFrame 생성
c = pd.DataFrame(closed_list)
print(c[['년도', '폐교연도', '정보공시 학교코드', '학교명']])

       년도  폐교연도   정보공시 학교코드        학교명
225  2014  2015  S010001256   서울흥일초등학교
422  2014  2015  S010002020  알로이시오초등학교
421  2017  2018  S010002022     은혜초등학교
146  2019  2020  S010001047   서울염강초등학교
16   2022  2023  S010000754   서울반포초등학교
454  2022  2023  S010002195   서울화양초등학교
234  2023  2024  S010001407   서울면중초등학교
510  2024  2025  S010002379   서울보광초등학교


In [6]:
c.to_csv("이미폐교.csv")

In [7]:
# 폐교 학교 코드 리스트
closed_codes = c['정보공시 학교코드'].unique()

# merged_data에서 폐교 학교 제외
merged_data_filtered = merge_data[~merge_data['정보공시 학교코드'].isin(closed_codes)]

print(merged_data_filtered.shape)

(10079, 35)


In [8]:
len(merged_data_filtered['정보공시 학교코드'].unique())

606

In [9]:
merged_data_filtered.isnull().sum()

년도              0
지역              0
정보공시 학교코드       0
학교명             0
입학생_남           1
입학생_여           1
입학생수            1
1학년 학급수         0
1학년 학생수         0
1학년 학급당 학생수     0
2학년 학급수         0
2학년 학생수         0
2학년 학급당 학생수     0
3학년 학급수         0
3학년 학생수         0
3학년 학급당 학생수     0
4학년 학급수         0
4학년 학생수         0
4학년 학급당 학생수     0
5학년 학급수         0
5학년 학생수         0
5학년 학급당 학생수     0
6학년 학급수         0
6학년 학생수         0
6학년 학급당 학생수     0
특수학급 학급수        0
특수학급 학생수        0
특수학급 학급당 학생수    0
학급수(계)          0
학생수(계)          0
학급당 학생수(계)      0
교사수             0
수업교원 1인당 학생수    0
위도              0
경도              0
dtype: int64

In [10]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# =======================
# 1. 데이터 준비
# =======================
df = merge_data[['년도','정보공시 학교코드','학교명','지역','입학생수','학생수(계)','교사수']]

# 학습용: 2024년까지
train = df[df['년도'] <= 2024].copy()
# 테스트용: 2025년
test  = df[df['년도'] == 2025].copy()

# =======================
# 2. 지역 원-핫 인코딩
# =======================
train = pd.get_dummies(train, columns=['지역'])
test  = pd.get_dummies(test, columns=['지역'])

# =======================
# 3. 내년도 타겟 생성 (shift)
# =======================
train_sorted = train.sort_values(['정보공시 학교코드','년도'])
for col in ['입학생수','학생수(계)','교사수']:
    train_sorted[col+'_next'] = train_sorted.groupby('정보공시 학교코드')[col].shift(-1)

# 내년도 값이 있는 행만 사용
train_clean = train_sorted.dropna(subset=['입학생수_next','학생수(계)_next','교사수_next'])

feature_cols = ['입학생수','학생수(계)','교사수'] + [c for c in train.columns if c.startswith('지역_')]
target_cols = ['입학생수_next','학생수(계)_next','교사수_next']

X_train = train_clean[feature_cols]
y_train = train_clean[target_cols]

# =======================
# 4. MultiOutput RandomForest 학습
# =======================
rf_multi = MultiOutputRegressor(RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1))
rf_multi.fit(X_train, y_train)

# =======================
# 5. 2025년 예측
# =======================
X_test = test[feature_cols]
y_test = test[['입학생수','학생수(계)','교사수']].values

y_pred = rf_multi.predict(X_test)

# =======================
# 6. 정확도 평가
# =======================
rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
mae  = mean_absolute_error(y_test, y_pred)

print("RandomForest (MultiOutput) → RMSE (입학생수, 학생수, 교사수):", rmse)
print("RandomForest (MultiOutput) → MAE (전체):", mae)

# =======================
# 7. 예측값 확인
# =======================
pred_df = test[['학교명','정보공시 학교코드']].copy()
pred_df[['입학생수_예측','학생수_예측','교사수_예측']] = y_pred
pred_df[['입학생수_실제','학생수_실제','교사수_실제']] = y_test
pred_df.head(10)

RandomForest (MultiOutput) → RMSE (입학생수, 학생수, 교사수): [10.48724051 49.10164229  3.50336978]
RandomForest (MultiOutput) → MAE (전체): 15.209584687734646


,학교명,정보공시 학교코드,입학생수_예측,학생수_예측,교사수_예측,입학생수_실제,학생수_실제,교사수_실제
9564,서울교육대학교부설초등학교,S000003511,96.145,611.065,31.325,96.0,611.0,31.0
9565,서울대학교사범대학부설초등학교,S000003563,81.930,531.400,33.305,87.0,570.0,36.0
9566,서울개원초등학교,S010000737,146.070,960.605,49.090,149.0,1006.0,51.0
9567,서울개일초등학교,S010000738,115.455,796.465,39.620,129.0,834.0,40.0
9568,서울개포초등학교,S010000739,112.930,856.735,49.120,115.0,886.0,51.0
9569,서울구룡초등학교,S010000741,74.130,671.950,36.035,70.0,561.0,32.0
9570,서울논현초등학교,S010000742,26.610,179.210,16.645,35.0,198.0,17.0
9571,서울대곡초등학교,S010000743,119.665,1230.850,52.920,108.0,1235.0,52.0
9572,서울대도초등학교,S010000744,211.505,1921.020,75.100,210.0,1956.0,73.0
9573,서울대모초등학교,S010000745,93.145,714.875,35.820,90.0,788.0,37.0


In [11]:
future_years = [2026, 2027, 2028, 2029,2030]
pred_list = []

# 초기 입력: 2025년 데이터
current_df = merge_data[merge_data['년도']==2025].copy()

for year in future_years:
    # 지역 원-핫 인코딩
    current_input = pd.get_dummies(current_df, columns=['지역'])
    
    # 피처 선택
    X_current = current_input[feature_cols]
    
    # 예측
    y_pred = rf_multi.predict(X_current)
    
    # 결과 저장
    pred_df = current_df[['정보공시 학교코드','학교명','지역','위도','경도']].copy()
    pred_df[['입학생수','학생수','교사수']] = y_pred
    pred_df['년도'] = year
    pred_list.append(pred_df)
    
    # 다음년도 입력 업데이트
    current_df[['입학생수','학생수(계)','교사수']] = y_pred

# 5년치 예측 데이터 결합
future_5yrs_df = pd.concat(pred_list, ignore_index=True)

In [12]:
future_5yrs_df

,정보공시 학교코드,학교명,지역,위도,경도,입학생수,학생수,교사수,년도
0,S000003511,서울교육대학교부설초등학교,서울특별시 서초구,37.490739,127.015424,96.145,611.065,31.325,2026
1,S000003563,서울대학교사범대학부설초등학교,서울특별시 종로구,37.577791,127.002875,81.930,531.400,33.305,2026
2,S010000737,서울개원초등학교,서울특별시 강남구,37.481618,127.058738,146.070,960.605,49.090,2026
3,S010000738,서울개일초등학교,서울특별시 강남구,37.486214,127.057742,115.455,796.465,39.620,2026
4,S010000739,서울개포초등학교,서울특별시 강남구,37.486805,127.069977,112.930,856.735,49.120,2026
...,...,...,...,...,...,...,...,...,...
3025,S010006432,서울항동초등학교,서울특별시 구로구,37.476731,126.823929,207.950,1235.840,63.515,2030
3026,S010006433,서울해누리초등학교,서울특별시 송파구,37.498364,127.101956,102.455,590.450,33.510,2030
3027,S010006476,서울양원숲초등학교,서울특별시 중랑구,37.608890,127.105540,121.300,783.155,43.530,2030
3028,S010006510,서울개현초등학교,서울특별시 강남구,37.481026,127.062043,129.450,791.795,36.385,2030


In [32]:
future_2030 = future_5yrs_df[future_5yrs_df['년도'] == 2030]
len(future_2030)
future_2030.to_csv("future_2030.csv",index=False)

In [13]:
len(future_5yrs_df['정보공시 학교코드'].unique())

606

In [14]:
# 수업교원 1인당 학생수 = 학생수(계) / 교사수
future_5yrs_df['수업교원 1인당 학생수'] = future_5yrs_df['학생수'] / future_5yrs_df['교사수']

In [15]:
# 2026 ~ 2030의 하위 5프로 학교의 기준
cols = ['입학생수','학생수','수업교원 1인당 학생수']
data_2026 = future_5yrs_df[future_5yrs_df['년도'] == 2026]
data_2026.groupby('지역')[cols].quantile(0.05)

,입학생수,학생수,수업교원 1인당 학생수
지역,,,
서울특별시 강남구,34.24825,226.41125,12.432460
서울특별시 강동구,46.24500,329.37400,13.697523
서울특별시 강북구,34.11725,231.06900,12.068085
서울특별시 강서구,19.85650,113.75250,9.342334
서울특별시 관악구,33.34125,210.28950,12.631835
서울특별시 광진구,32.32500,207.51500,12.538671
서울특별시 구로구,47.24750,276.89500,12.584872
서울특별시 금천구,23.39950,150.25200,9.348464
서울특별시 노원구,26.85850,163.81200,10.036212


연도별로 데이터를 나누고

지역별로 각 지표가 하위 5% 이하인 학교를 이상치로 판정

세 지표 모두 하위 5%인 학교만 선택 → 폐교 후보

이미 이전 연도에 선정된 폐교 후보는 제외

연도별 후보를 합쳐 최종 리스트 생성

In [16]:
years = [2026, 2027, 2028, 2029, 2030]
all_outlier_schools = []
already_closed_codes = set()  # 이전 연도에 폐교된 학교 코드 저장

cols = ['입학생수','학생수','수업교원 1인당 학생수']

for year in years:
    # 해당 연도 데이터 선택
    data_year = future_5yrs_df[future_5yrs_df['년도'] == year].copy()
    
    # 이미 폐교된 학교 제외
    data_year = data_year[~data_year['정보공시 학교코드'].isin(already_closed_codes)]
    
    # 이상치 판정용 DataFrame 초기화
    outlier_flags = pd.DataFrame(False, index=data_year.index, columns=cols)
    
    # 각 컬럼별 지역 기준 5% 이하 판정
    for col in cols:
        # 지역별 5% 분위수 계산
        q10_by_region = data_year.groupby('지역')[col].transform(lambda x: x.quantile(0.05))
        # 5% 이하인 경우 True
        outlier_flags[col] = data_year[col] <= q10_by_region
    
    # 세 컬럼 모두 10% 이하인 학교 선택
    outlier_schools = data_year[outlier_flags.all(axis=1)].copy()
    outlier_schools['년도'] = year  # 연도 표시
    
    # 이미 폐교된 학교 코드 업데이트
    already_closed_codes.update(outlier_schools['정보공시 학교코드'].tolist())
    
    # 리스트에 추가
    all_outlier_schools.append(outlier_schools)

# 2026~2029년 폐교 후보 합치기
outlier_schools_2026_2029 = pd.concat(all_outlier_schools, ignore_index=True)

In [17]:
outlier_schools_2026_2029['폐교가능성년도'] = outlier_schools_2026_2029['년도']
outlier_schools_2026_2029.to_csv("폐교학교.csv", index = False, encoding = 'utf-8-sig')

In [18]:
len(outlier_schools_2026_2029['정보공시 학교코드'])

90

In [19]:
data1 = pd.read_csv("./data/2025년도_입학생 현황(초)_서울특별시교육청.csv")
data2 = data1[['정보공시 학교코드','학교명','지역']]
data2

,정보공시 학교코드,학교명,지역
0,S000003511,서울교육대학교부설초등학교,서울특별시 서초구
1,S000003563,서울대학교사범대학부설초등학교,서울특별시 종로구
2,S010000737,서울개원초등학교,서울특별시 강남구
3,S010000738,서울개일초등학교,서울특별시 강남구
4,S010000739,서울개포초등학교,서울특별시 강남구
...,...,...,...
601,S010006432,서울항동초등학교,서울특별시 구로구
602,S010006433,서울해누리초등학교,서울특별시 송파구
603,S010006476,서울양원숲초등학교,서울특별시 중랑구
604,S010006510,서울개현초등학교,서울특별시 강남구


In [20]:
final_data = data2.merge(
    outlier_schools_2026_2029[['정보공시 학교코드','폐교가능성년도']],
    on = '정보공시 학교코드',
    how = 'left'
)

len(final_data)

606

In [21]:
final_data

,정보공시 학교코드,학교명,지역,폐교가능성년도
0,S000003511,서울교육대학교부설초등학교,서울특별시 서초구,NaN
1,S000003563,서울대학교사범대학부설초등학교,서울특별시 종로구,NaN
2,S010000737,서울개원초등학교,서울특별시 강남구,NaN
3,S010000738,서울개일초등학교,서울특별시 강남구,NaN
4,S010000739,서울개포초등학교,서울특별시 강남구,NaN
...,...,...,...,...
601,S010006432,서울항동초등학교,서울특별시 구로구,NaN
602,S010006433,서울해누리초등학교,서울특별시 송파구,NaN
603,S010006476,서울양원숲초등학교,서울특별시 중랑구,NaN
604,S010006510,서울개현초등학교,서울특별시 강남구,NaN


In [22]:
final_data['폐교가능성년도'] = final_data['폐교가능성년도'].fillna(0).astype(int)

In [23]:
len(final_data)

606

In [24]:
location_data = final_data.merge(
location, on ='정보공시 학교코드', how = 'inner'
)

In [25]:
location_data

,정보공시 학교코드,학교명,지역,폐교가능성년도,위도,경도
0,S000003511,서울교육대학교부설초등학교,서울특별시 서초구,0,37.490739,127.015424
1,S000003563,서울대학교사범대학부설초등학교,서울특별시 종로구,0,37.577791,127.002875
2,S010000737,서울개원초등학교,서울특별시 강남구,0,37.481618,127.058738
3,S010000738,서울개일초등학교,서울특별시 강남구,0,37.486214,127.057742
4,S010000739,서울개포초등학교,서울특별시 강남구,0,37.486805,127.069977
...,...,...,...,...,...,...
601,S010006432,서울항동초등학교,서울특별시 구로구,0,37.476731,126.823929
602,S010006433,서울해누리초등학교,서울특별시 송파구,0,37.498364,127.101956
603,S010006476,서울양원숲초등학교,서울특별시 중랑구,0,37.608890,127.105540
604,S010006510,서울개현초등학교,서울특별시 강남구,0,37.481026,127.062043


In [26]:
import folium
import numpy as np

# Haversine 함수: 두 위도/경도 사이 거리(m) 계산
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # 지구 반지름 (m)
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda/2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

# 반경 500m 내 학교 수를 저장할 리스트
nearby_counts = []

# 지도 초기화
m = folium.Map(location=[37.5665, 126.9780], zoom_start=12)

for idx, school in location_data.iterrows():
    lat1, lon1 = school['위도'], school['경도']
    
    # 자기 자신 제외하고 모든 학교와 거리 계산
    distances = location_data.apply(
        lambda row: haversine(lat1, lon1, row['위도'], row['경도'])
                    if row['정보공시 학교코드'] != school['정보공시 학교코드'] else np.inf,
        axis=1
    )
    
    # 500m 내 학교 수 계산
    nearby_count = (distances <= 500).sum()
    nearby_counts.append(nearby_count)
    
    # 색상 결정
    if school['폐교가능성년도'] == 0:
        color = 'blue'
    else:
        color = 'red' if nearby_count > 0 else 'yellow'
    
    folium.CircleMarker(
        location=[lat1, lon1],
        radius=5,
        color=color,
        fill=True,
        fill_opacity=0.7,
        popup=f"{school['학교명']} ({school['폐교가능성년도']}), 주변 학교: {nearby_count}"
    ).add_to(m)

# 컬럼 추가
location_data['500m내_학교수'] = nearby_counts

# 지도 저장
m.save("seoul_schools_map.html")


In [27]:
# 빨간색 마커 개수 계산
red_count = len(location_data[
    (location_data['폐교가능성년도'] != 0) & 
    (location_data['500m내_학교수'] > 0)
])
# 노란색 마커 개수 계산
yellow_count = len(location_data[
    (location_data['폐교가능성년도'] != 0) & 
    (location_data['500m내_학교수'] == 0)
])

print("노란색 마커 개수:", yellow_count)


print("빨간색 마커 개수:", red_count)


노란색 마커 개수: 48
빨간색 마커 개수: 42


In [33]:
location_data[
    (location_data['폐교가능성년도'] != 0) & 
    (location_data['500m내_학교수'] > 0)
]

,정보공시 학교코드,학교명,지역,폐교가능성년도,위도,경도,500m내_학교수
34,S010000771,서울양전초등학교,서울특별시 강남구,2027,37.490292,127.069378,1
101,S010000948,서울천일초등학교,서울특별시 강동구,2026,37.544361,127.133439,2
105,S010000952,서울풍납초등학교,서울특별시 송파구,2029,37.535086,127.117867,1
121,S010001017,서울등명초등학교,서울특별시 강서구,2028,37.560880,126.842329,1
124,S010001020,서울등원초등학교,서울특별시 강서구,2030,37.560326,126.850073,2
157,S010001054,서울은정초등학교,서울특별시 양천구,2028,37.511129,126.867046,1
158,S010001055,서울정곡초등학교,서울특별시 강서구,2028,37.576960,126.818934,1
176,S010001208,서울금천초등학교,서울특별시 금천구,2026,37.454978,126.912769,2
186,S010001218,서울도신초등학교,서울특별시 영등포구,2026,37.499269,126.900713,1
207,S010001239,서울영남초등학교,서울특별시 금천구,2030,37.475435,126.909864,2


In [28]:
yellow_schools = final_data[
    (final_data['폐교가능성년도'] != 0)
]

print("중복 제거 전:", len(yellow_schools))
print("중복 제거 후:", yellow_schools['정보공시 학교코드'].nunique())


중복 제거 전: 90
중복 제거 후: 90
